# Data Loading and Cleaning

In [1]:
# imports
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [57]:
# Loading csv with preprocessed data
df = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data.csv')

# Cleaning the data
# Removing zero ratings
df = df[df['rating'] != 0]

In [58]:
# Removing image_url columns because we will not use it.
df = df.drop(['img_s'], axis=1)
df = df.drop(['img_m'], axis=1)
df = df.drop(['img_l'], axis=1)
df = df.dropna()

In [60]:
df.head()

Unnamed: 0  user_id                   location      age        isbn  \
1            1        8   timmins, ontario, canada  34.7439  0002005018   
5            5    67544   toronto, ontario, canada  30.0000  0002005018   
9            9   123629  kingston, ontario, canada  34.7439  0002005018   
11          11   200273    comber, ontario, canada  34.7439  0002005018   
12          12   210926    guelph, ontario, canada  34.7439  0002005018   

    rating    book_title           book_author  year_of_publication  \
1        5  Clara Callan  Richard Bruce Wright               2001.0   
5        8  Clara Callan  Richard Bruce Wright               2001.0   
9        9  Clara Callan  Richard Bruce Wright               2001.0   
11       8  Clara Callan  Richard Bruce Wright               2001.0   
12       9  Clara Callan  Richard Bruce Wright               2001.0   

                publisher                                            Summary  \
1   HarperFlamingo Canada  In a small town in Canada, Clara Callan reluct...   
5   HarperFlamingo Canada  In a small town in Canada, Clara Callan reluct...   
9   HarperFlamingo Canada  In a small town in Canada, Clara Callan reluct...   
11  HarperFlamingo Canada  In a small town in Canada, Clara Callan reluct...   
12  HarperFlamingo Canada  In a small town in Canada, Clara Callan reluct...   

   Language       Category      city    state country  
1        en  ['Actresses']   timmins  ontario  canada  
5        en  ['Actresses']   toronto  ontario  canada  
9        en  ['Actresses']  kingston  ontario  canada  
11       en  ['Actresses']    comber  ontario  canada  
12       en  ['Actresses']    guelph  ontario  canada

In [61]:
df.to_csv('Preprocessed_data_cleaned.csv', index=False)

In [20]:
# From now on we can work with df or load from the beggining 
# the Preprocessed_data_cleaned.csv file

-------------------------------------------------------------------
# Demographic Filtering

In [2]:
# Loading csv with preprocessed data cleaned
# Here add your path to the Preprocessed_data_cleaned.csv file!
# You just need to run the first and this cell from now on everytime you want to run
# the code
df_cleaned = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data_cleaned.csv')

In [3]:
df_cleaned.head(-1)

Unnamed: 0  user_id                      location      age  \
0                1        8      timmins, ontario, canada  34.7439   
1                5    67544      toronto, ontario, canada  30.0000   
2                9   123629     kingston, ontario, canada  34.7439   
3               11   200273       comber, ontario, canada  34.7439   
4               12   210926       guelph, ontario, canada  34.7439   
...            ...      ...                           ...      ...   
359465     1031163   278843  pismo beach, california, usa  28.0000   
359466     1031169   278851            dallas, texas, usa  33.0000   
359467     1031171   278851            dallas, texas, usa  33.0000   
359468     1031172   278851            dallas, texas, usa  33.0000   
359469     1031173   278851            dallas, texas, usa  33.0000   

              isbn  rating                                         book_title  \
0       0002005018       5                                       Clara Callan   
1       0002005018       8                                       Clara Callan   
2       0002005018       9                                       Clara Callan   
3       0002005018       8                                       Clara Callan   
4       0002005018       9                                       Clara Callan   
...            ...     ...                                                ...   
359465  0743525493       7  The Motley Fool's What To Do with Your Money N...   
359466  067161746X       7  The Bachelor Home Companion: A Practical Guide...   
359467  0767907566       5  All Elevations Unknown: An Adventure in the He...   
359468  0884159221       7  Why stop?: A guide to Texas historical roadsid...   
359469  0912333022       7  The Are You Being Served? Stories: 'Camping In...   

                 book_author  year_of_publication               publisher  \
0       Richard Bruce Wright               2001.0   HarperFlamingo Canada   
1       Richard Bruce Wright               2001.0   HarperFlamingo Canada   
2       Richard Bruce Wright               2001.0   HarperFlamingo Canada   
3       Richard Bruce Wright               2001.0   HarperFlamingo Canada   
4       Richard Bruce Wright               2001.0   HarperFlamingo Canada   
...                      ...                  ...                     ...   
359465         David Gardner               2002.0  Simon & Schuster Audio   
359466         P.J. O'Rourke               1987.0            Pocket Books   
359467          Sam Lightner               2001.0          Broadway Books   
359468         Claude Dooley               1985.0         Lone Star Books   
359469          Jeremy Lloyd               1997.0              Kqed Books   

                                                  Summary Language  \
0       In a small town in Canada, Clara Callan reluct...       en   
1       In a small town in Canada, Clara Callan reluct...       en   
2       In a small town in Canada, Clara Callan reluct...       en   
3       In a small town in Canada, Clara Callan reluct...       en   
4       In a small town in Canada, Clara Callan reluct...       en   
...                                                   ...      ...   
359465                                                  9        9   
359466  A tongue-in-cheek survival guide for single pe...       en   
359467  A daring twist on the travel-adventure genre t...       en   
359468                                                  9        9   
359469  These hilarious stories by the creator of publ...       en   

             Category         city       state country  
0       ['Actresses']      timmins     ontario  canada  
1       ['Actresses']      toronto     ontario  canada  
2       ['Actresses']     kingston     ontario  canada  
3       ['Actresses']       comber     ontario  canada  
4       ['Actresses']       guelph     ontario  canada  
...               ...          ...         ...     ...  
359465              9  pismo 

We will start using the weighted rating (WR) that IMDB is using, which is given as: 

Weighted Rating(WR)=(v/(v+m))*R)+((m/(v+m))*C)

where: 

* v is the number of votes per book
* m is the minimum votes required to be listed in the chart
* R is the average rating of the book
* C is the mean vote accrose the whole dataframe

Thus, we need to load and clean from zeros the BX-Book ratings dataframe and make the vote count to see how many users have voted for the same book.

In [26]:
# Loading csv with book rating data
df_book_rating = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BX-Book-Ratings.csv', sep=';', encoding= 'unicode_escape', engine='python')

In [27]:
df_book_rating.head(5)

User-ID          ISBN Book-Rating
0  '276725'  '034545104X'         '0'
1  '276726'  '0155061224'         '5'
2  '276727'  '0446520802'         '0'
3  '276729'  '052165615X'         '3'
4  '276729'  '0521795028'         '6'

In [30]:
df_book_rating.keys()

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [43]:
# I wanna turn the last column (Book ratings) to int!!!! :):):)::):) 
pd.to_numeric(df_book_rating['Book-Rating'])

ValueError: Unable to parse string "'0'" at position 0

-----------------------------------------------------------
# Content Filtering

In [63]:
# Loading csv with preprocessed data cleaned
# Here add your path to the Preprocessed_data_cleaned.csv file!
# You just need to run the first and this cell from now on everytime you want to run
# the code
df_cleaned = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data_cleaned.csv')

--------------------------------------------------------------------------------
# Collaborative Filtering

In [ ]:
# Loading csv with preprocessed data cleaned
# Here add your path to the Preprocessed_data_cleaned.csv file!
# You just need to run the first and this cell from now on everytime you want to run
# the code
df_cleaned = pd.read_csv('/Users/mep/Desktop/mep/CompTools/BookRecommendationSystem/Preprocessed_data_cleaned.csv')